In [6]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from bs4 import *

In [7]:
filename = 'sample_data.txt'
data = open(filename,'r')
soup = BeautifulSoup(data,'lxml')

<objectdata version="1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nonamespaceschemalocation="Logging.xsd"><devicename>0089PS06CT1</devicename><deviceid>32</deviceid><timestamp>2017-07-07T23:00:05.633</timestamp><incr>21126</incr><tokenid>0089PS06CT12017-07-07T23:00:03417</tokenid><seqnb>5864</seqnb><general errornb="0" ie="15589" iostate="8800" is="14764" lftf="N" lfts="7" oc="0" oi="269396" on="417" ox="5136"><timestamp>2017-07-07T23:00:03.593</timestamp><otl unit="inch"><value>33.34</value></otl><tt unit="ms"><value>309</value></tt><oga unit="inch"><value>314.13</value></oga><devicelist>FC300008</devicelist><udi udi2="1116" udi3="1" udi4="0" udi5="1024"></udi><uds uds1="&lt;rxstring&gt;07/07/2017;23:56:44;0111.6;LB;13;3402;999;0002794051&lt;/rxstring&gt;&lt;rxoffset&gt;4.80&lt;/rxoffset&gt;&lt;rxfields&gt;&lt;ss&gt;13&lt;/ss&gt;&lt;sw&gt;0111.6&lt;/sw&gt;&lt;/rxfields&gt;&lt;condition&gt;OcsRx,OcsValid&lt;/condition&gt;&lt;txfields&gt;&lt;ss&gt;13&lt;/ss&gt;&lt;sw&g

In [3]:
#Capture all field names
obj = soup.objectdata
features = {n.name:[] for n in obj}
#Grab field information for each field
for obj in soup.find_all('objectdata'):
    for feature in features.keys():
        features[feature].append(obj.find(feature))
df = pd.DataFrame(features)
df.head(2)

,barcode,condition,deviceid,devicename,general,hostmessage,incr,ocsdata,scaledata,seqnb,sorterstate,sortstate,timestamp,tokenid,volumetric
0,"<barcode cc=""1"" vcc=""1""><codevalid id=""r""><st>...","<condition>NoRead,ValidDim,NotLFT</condition>",<deviceid>32</deviceid>,<devicename>0089PS06CT1</devicename>,"<general errornb=""0"" ie=""15589"" iostate=""8800""...",<hostmessage>PDF</hostmessage>,<incr>21126</incr>,<ocsdata><rxstring>07/07/2017;23:56:44;0111.6;...,"<scaledata ows=""19""><owe unit=""LB""><value>111....",<seqnb>5864</seqnb>,"<sorterstate state=""started""><speed unit=""ft/m...","<sortstate session=""SOS"" sortname=""2017-07-07 ...",<timestamp>2017-07-07T23:00:05.633</timestamp>,<tokenid>0089PS06CT12017-07-07T23:00:03417</to...,"<volumetric oms1=""0000"" oms2=""00000000"" oms3=""..."
1,"<barcode cc=""1"" vcc=""1""><codevalid id=""j""><st>...","<condition>ValidDim,ValidWeight,ValidRead,PDFN...",<deviceid>32</deviceid>,<devicename>0089PS06CT1</devicename>,"<general errornb=""0"" ie=""16743"" iostate=""8800""...",<hostmessage>]C0037962200190000357853670078708...,<incr>22266</incr>,<ocsdata><rxstring>07/07/2017;23:56:45;0007.2;...,"<scaledata ows=""0""><owe unit=""LB""><value>7.20<...",<seqnb>5865</seqnb>,"<sorterstate state=""started""><speed unit=""ft/m...","<sortstate session=""SOS"" sortname=""2017-07-07 ...",<timestamp>2017-07-07T23:00:06.061</timestamp>,<tokenid>0089PS06CT12017-07-07T23:00:04418</to...,"<volumetric oms1=""0000"" oms2=""00000000"" oms3=""..."


In [4]:
"""Pre-process label for each example"""
#Create dictionary to assign encoded integer to each class.
classes = ['LFT','TooBig','NoRead','ValidDim','MultiRead','Irreg','TooSmall','Gap']
class_dict = {classes[i]:i for i in range(len(classes))}

#Extract and encode contents of condition field only for conditions listed in the `classes` list
conditions = [[class_dict[c] for c in obj.condition.contents[0].split(',') if c in classes]\
              for obj in soup.find_all('objectdata')]
##Perform one-hot-encoding
#Initialize matrix
conditions_encoded = np.zeros([len(conditions),len(classes)])
#Encode position in matrix for place of class
for c in range(len(conditions)):
    conditions_encoded[c,conditions[c]] = 1
#Store in dataframe
label_df = pd.DataFrame(conditions_encoded,columns=classes).astype(int)

In [33]:
"""Extract relevant information from each field"""
#Time stamp
[date,time] = obj.timestamp.contents[0].split('T')
#Gap information
gap = float(obj.oga.contents[0].contents[0])
#Item volume
vol_keys = ['ohe','owi','ole']
[height,width,length] = [float(obj.volumetric.size.attrs[key]) for key in vol_keys]
vol = height*width*length
vol_units = obj.volumetric.size.attrs['unit']
#Item angle
angle = int(obj.volumetric.oa.value.contents[0])
#Item Velocity
vel = int(obj.volumetric.otve.value.contents[0])
vel_units = obj.volumetric.otve.attrs['unit']
#Item weight
weight = obj.scaledata

#Conveyor belt velocity
belt_vel = float(obj.sorterstate.speed.value.contents[0])
belt_vel_units = obj.sorterstate.speed.attrs['unit']
float(obj.scaledata.value.contents)

TypeError: float() argument must be a string or a number, not 'list'